In [ ]:
import hack
from DatasetLoader import *
from DataGenerator import *
import pickle
import numpy as np
import os 
import matplotlib.animation as manimation
from IPython.display import clear_output
from sklearn.metrics import mean_absolute_error

In [ ]:
with open('../Data/DATASET_REAL/real_world_trajectories.pickle', 'rb') as handle:
     real_world_trajectories =pickle.load(handle)
with open('../Data/DATASET_REAL/real_world_relations.pickle', 'rb') as handle:
     real_world_relations= pickle.load(handle)
with open('../Models/Dataset_Simple_scaler.pickle.pickle', 'rb') as handle:
    dataset_scaler = pickle.load(handle)

In [ ]:
from Test import *
# Sample Real World Traj Video
traj_ind=-1
if traj_ind>=0:
    xy= real_world_trajectories[traj_ind][0,:,:,2:4]
    r = real_world_trajectories[traj_ind][0,0,:,0]
    edge =  real_world_relations[traj_ind][0]
    make_video(xy,r,edge,'test.mp4')

In [ ]:
from Networks import *
Tpns = TemporalPropagationNetwork()
Tpn = Tpns.getModel(10,6,1,100,50)

Pns= PropagationNetwork()
Pn1=Pns.getModel(10,6,1)

Pns.setModel(10,'../Models/PN_fixed.hdf5') 
Tpn.load_weights(('../Models/TPN_fixed.hdf5'))

In [ ]:
from Networks import *
results=list()
for traj_ind in tqdm(range(102)):
    traj=real_world_trajectories[traj_ind]
    rels=real_world_relations[traj_ind]
    Tpn = Tpns.getModel(traj.shape[2],6,1,traj.shape[1],0)
    results.append(Test_TPN(traj,rels,Tpn,dataset_scaler))

In [ ]:
# Finding Contact timesteps.
relations_timesteps=np.zeros((102,6))
for traj_ind in tqdm(range(102)):
    isended=False
    traj=real_world_trajectories[traj_ind]    
    for ts in range(traj.shape[1]):
        if isended:
            break
        for obj2 in range(1,traj.shape[2]):
            if np.linalg.norm(traj[0,ts,0,2:4]-traj[0,ts,obj2,2:4])<(traj[0,ts,0,0]+traj[0,ts,obj2,0])/2+0.01:
                for ts_add in range(5):
                    relations_timesteps[traj_ind,1+ts_add]=ts+ts_add
                isended=True
                break

In [ ]:
def custom_Test(traj,relations,startStep,Ins,scaler):
    n_objects = traj.shape[2]
    n_of_rel=1
    n_relations=n_objects*(n_objects-1)
    In=Ins.getModel(n_objects)
    GroundData=traj
    n_of_traj=GroundData.shape[0]
    frame_len=    traj.shape[1]  -startStep         
    xy_origin_pos=copy.deepcopy(GroundData[:,startStep:,:,2:4]);
    xy_origin_vel=copy.deepcopy(GroundData[:,startStep:,:,4:6]);
    
    dataToModel= np.zeros([n_of_traj,frame_len,n_objects,6])
    dataToModel[:,0,:,:]= copy.deepcopy(GroundData[:,startStep,:,:])
    dataToModel[:,:,0,:]= copy.deepcopy(GroundData[:,startStep:,0,:])
    dataToModel[:,:,:,:2]= copy.deepcopy(GroundData[:,startStep:,:,:2])
    dataToModel[:,0,1:,4:6]= 0
    r= dataToModel[:,0,:,0]
    dataToModel=scaler.transform(dataToModel)
    val_receiver_relations = np.zeros((n_of_traj, n_objects, n_relations), dtype=float);
    val_sender_relations   = np.zeros((n_of_traj, n_objects, n_relations), dtype=float);
    val_relation_info = relations#np.zeros((n_of_traj, n_relations, num_of_rel_type))
    propagation = np.zeros((n_of_traj, n_objects, 100))
    cnt = 0
    for m in range(n_objects):
        for j in range(n_objects):
            if(m != j):
                inzz=np.linalg.norm(dataToModel[:,0,m,2:4]-dataToModel[:,0,j,2:4],axis=1)<scaler.relation_threshold
                val_receiver_relations[inzz, j, cnt] = 1.0
                val_sender_relations[inzz, m, cnt]   = 1.0
                cnt += 1
    edges=val_relation_info
    for i in range(1,frame_len):
        velocities=In.predict({'objects': dataToModel[:,i-1,:,:],'sender_relations': val_sender_relations,'receiver_relations': val_receiver_relations,'relation_info': val_relation_info,'propagation':propagation})
        dataToModel[:,i,1:,2:4]=dataToModel[:,i-1,1:,2:4]
        dataToModel[:,i,1:,4:6]=velocities[:,:,:]; 
        dataToModel[:,i,1:,:]=PositionCalculateNext(dataToModel[:,i,1:,:],scaler)
        val_receiver_relations = np.zeros((n_of_traj, n_objects, n_relations), dtype=float);
        val_sender_relations   = np.zeros((n_of_traj, n_objects, n_relations), dtype=float);
        cnt = 0
        for m in range(n_objects):
            for j in range(n_objects):
                if(m != j):
                    inzz=np.linalg.norm(dataToModel[:,i,m,2:4]-dataToModel[:,i,j,2:4],axis=1)<scaler.relation_threshold
                    val_receiver_relations[inzz, j, cnt] = 1.0
                    val_sender_relations[inzz, m, cnt]   = 1.0                    
                    cnt += 1
    pred_xy = scaler.inv_transform(dataToModel)
    
    xy_calculated_pos=pred_xy[:,:,:,2:4]
    xy_calculated_vel=pred_xy[:,:,:,4:6]
    return mean_squared_error(xy_calculated_pos[:,-1,1:,:].reshape(-1,2),xy_origin_pos[:,-1,1:,:].reshape(-1,2))

In [ ]:
from sklearn.metrics import mean_squared_error 
errors=dict()
for traj_ind in tqdm(range(102)):
    Pn=Pns.getModel(real_world_trajectories[traj_ind].shape[2],6,1)
    traj_to_give=real_world_trajectories[traj_ind]
    rels_to_give=results[traj_ind][3]
    
    rels_to_give[rels_to_give>0.7]=1
    rels_to_give[rels_to_give<0.8]=0
    
    for ts_ind in range(6):
        ts=int(relations_timesteps[traj_ind,ts_ind])
        errors[traj_ind,ts_ind]=custom_Test(traj_to_give,rels_to_give[:,:,ts,:],ts,Pns,dataset_scaler)